In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


In [2]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
)

In [3]:
from langchain.tools import BaseTool


class EvaluateMathExpression(BaseTool):
    name = "Math Evaluation"
    description = 'use this tool to evaluate a math expression.'

    def _run(self, expr: str):
        return eval(expr)

    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")


tools = [EvaluateMathExpression()]


In [4]:
from langchain.agents import create_react_agent, AgentExecutor

# create_react_agent
# create_json_agent
# create_structured_chat_agent
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=qw_llm_openai,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)
prompt = ''
agent2 = create_react_agent(qw_llm_openai, tools, prompt)
agent_executor = AgentExecutor(agent=agent2, tools=tools)
agent_executor.invoke({"input": "hi"})

# Use with chat history
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name?",
        # Notice that chat_history is a string
        # since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: My name is Bob\nAI: Hello Bob!",
    }
)

/Users/pangmengting/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
for message in agent.agent.llm_chain.prompt.messages:
    print(message)

prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is

In [6]:
agent(f"What is 2 * 2 * 0.13 - 1.001?")

/Users/pangmengting/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Math Evaluation",
    "action_input": "2*2*0.13-1.001"
}
```
Observation: -0.48099999999999987
Thought:```json
{
    "action": "Final Answer",
    "action_input": "-0.481"
}
```

> Finished chain.


{'input': 'What is 2 * 2 * 0.13 - 1.001?',
 'chat_history': [],
 'output': '-0.481'}

In [7]:
# 修改内置的prompt
from langchain.agents.conversational_chat.prompt import (PREFIX)

system_message = PREFIX + "\n" + '''
Unfortunately, Assistant is terrible at maths. Assistant should always refers to available tools and never try to answer math questions by itself
'''

In [8]:
new_prompt = agent.agent.create_prompt(
    system_message=system_message,
    tools=tools
)

agent.agent.llm_chain.prompt = new_prompt

In [9]:
agent(f"What is 2 * 2 * 0.13 - 1.001?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Math Evaluation",
    "action_input": "2*2*0.13 - 1.001"
}
```
Observation: -0.48099999999999987
Thought:```json
{
    "action": "Final Answer",
    "action_input": "-0.481"
}
```

> Finished chain.


{'input': 'What is 2 * 2 * 0.13 - 1.001?',
 'chat_history': [HumanMessage(content='What is 2 * 2 * 0.13 - 1.001?'),
  AIMessage(content='-0.481')],
 'output': '-0.481'}

In [10]:
agent.memory.buffer

[HumanMessage(content='What is 2 * 2 * 0.13 - 1.001?'),
 AIMessage(content='-0.481'),
 HumanMessage(content='What is 2 * 2 * 0.13 - 1.001?'),
 AIMessage(content='-0.481')]